# Retrieval Chain and Document Chain

## Data Ingestion

In [ ]:
# Loading data from website
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='\n\n\n\n\nGet started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and w

## Data Splitting

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splitted_docs = splitter.split_documents(docs)
splitted_docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transforma

## Embedding and Vector Store

In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model='llama3.2:1b')
vectorStoreDb = FAISS.from_documents(splitted_docs, embeddings)

In [6]:
# Querying vector store db
query = "LangSmith is a platform for building production-grade LLM applications."
vectorStoreDb.similarity_search(query)

[Document(id='82416d00-b814-4086-8964-c6c55a6dcbc2', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nLangSmith is a standalone platform that can be used on it's own no matter how you're creating your LLM applicatons."),
 Document(id='69d3bde3-a580-4ba3-b7b0-1e31620d844c', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applicatio

## Document Chain 

In [7]:
#  Create OpenAI LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [8]:
# Creating document chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
<context>
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n<context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000217F7D35240>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000217F788E710>, root_client=<openai.OpenAI object at 0x00000217F7D35D80>, root_async_client=<openai.AsyncOpenAI object at 0x00000217F7D348E0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_do

In [13]:
# Querying document chain using a manually created context
from langchain_core.documents import Document

document_chain.invoke({
    "input": 'What is Langchain used for?',
    'context': [Document(page_content="""LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. With LangSmith you can:

Trace LLM Applications: Gain visibility into LLM calls and other parts of your application's logic.
Evaluate Performance: Compare results across models, prompts, and architectures to identify what works best.
Improve Prompts: Quickly refine prompts to achieve more accurate and reliable results.""")]
})

'What is LangSmith used for? \n\nLangSmith is a platform for building production-grade LLM applications, allowing users to monitor and evaluate their applications, trace LLM calls, evaluate performance, and improve prompts for better results.'

## Retrieval Chain

In [15]:
# Creating retriever
from langchain.chains import create_retrieval_chain

retriever = vectorStoreDb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
# Querying retrieval chain

retrieval_chain.invoke({
    'input': 'What is langchain used for?'
})

{'input': 'What is langchain used for?',
 'context': [Document(id='82416d00-b814-4086-8964-c6c55a6dcbc2', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nLangSmith is a standalone platform that can be used on it's own no matter how you're creating your LLM applicatons."),
  Document(id='69d3bde3-a580-4ba3-b7b0-1e31620d844c', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a